>## Aplicação 05 - Housing

O housing dataset possui dados sobre casas. Os objetivos deste notebook são prever, utilizando SVM, o preço das casas utilizando no máximo dois parâmetros como base.
O dataset foi obtido em: https://www.kaggle.com/datasets/camnugent/california-housing-prices

In [1]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

#leitura do csv
dataset = './datasets/housing.csv'
data = pd.read_csv(dataset)


In [2]:
# Visualização do dataset
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
# Informações dos dados
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [4]:
# O rótulo ocean_proximity apresenta dados não numéricos (descritivos)
data['ocean_proximity'].unique()

array(['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND'],
      dtype=object)

#### Lidando com os dados descritivos

In [5]:
# Criando o encoder para os dados de ocean_proximity
lb = LabelEncoder()
encoder = lb.fit(data['ocean_proximity'])

# Alterando o dataframe com a nova codificação
data['ocean_proximity'] = encoder.fit_transform(data['ocean_proximity'])
data.head()
                                                

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,3
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,3
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,3
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,3
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,3


#### Limpando os dados nulos

In [26]:
# faz com que as colunas que possuem mais de 10% dos dados nulos sejam armazenadas em valores_nulos
valores_nulos = data.columns[(data.isnull().sum() / data.shape[0]) > 0.1]

Index([], dtype='object')

In [28]:
data = data.drop(valores_nulos)

In [29]:
# Verifica a quantidade de valores nulos ainda presentes (mesmo após eliminar as colunas com mais de 10%)
data.isnull().sum().sort_values(ascending=False)

total_bedrooms        207
longitude               0
latitude                0
housing_median_age      0
total_rooms             0
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

>Ainda temos dados vazios em total_bedrooms. Uma boa pratica é substituir esses dados ausentes pelo valor -1

In [30]:
# o método fillna(x) substitui tudo que for null pelo valor entre parênteses
data = data.fillna(-1)

In [31]:
data.isnull().sum().sort_values(ascending=False)

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

#### Separando os daos de Treino e Teste

In [32]:
# X é o conjunto de dados sem o parâmetro que queremos prever
# y é o conjunto de dados alvo (queremos preve-lo)
X = data.drop('median_house_value', axis=1)
y = data.median_house_value

In [33]:
# Divisão dos dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

> Agora todos os valores estão tratados e é possível começar a selecionar os algoritmos para criação do modelo de previsão.

#### Selecionando os Algoritmos

> #### Regressão Linear


In [34]:
from sklearn.linear_model import LinearRegression

In [35]:
# Criando o regressor com os dados de treino
r_linear = LinearRegression().fit(X_train,y_train)

In [36]:
# Avaliando o regressor com os dados de teste
y_pred = r_linear.predict(X_test)

> #### Arvore de Regressão

In [37]:
from sklearn import tree

In [41]:
r_arvore = tree.DecisionTreeRegressor(random_state=42).fit(X_train,y_train)

In [42]:
y_pred_arvore = r_arvore.predict(X_test)

> ### KNN

In [43]:
from sklearn.neighbors import KNeighborsRegressor

In [46]:
r_knn = KNeighborsRegressor(n_neighbors=2).fit(X_train,y_train)

In [47]:
y_pred_knn = r_knn.predict(X_test)

### Avaliando os modelos criados

> ### Erro Médio Quadráticos (RMS) e Erro Médio Absoluto

In [48]:
from sklearn.metrics import mean_absolute_error

In [50]:
from sklearn.metrics import mean_squared_error

In [53]:
# Regressão
print(mean_absolute_error(y_test,y_pred))
print(mean_squared_error(y_test,y_pred))

52628.98063301301
5251270210.630704


In [55]:
# Árvore
print(mean_absolute_error(y_test,y_pred_arvore))
print(mean_squared_error(y_test,y_pred_arvore))

44061.62524224806
4817251502.855863


In [56]:
# KNN
print(mean_absolute_error(y_test,y_pred_knn))
print(mean_squared_error(y_test,y_pred_knn))

86142.76768410853
12615998415.737646
